In [ ]:
df['pf_ratio'] = 0
pf_ratio_is_na = df.po2.isna() | df.fio2.isna()
df.loc[~pf_ratio_is_na, 'pf_ratio'] = df.loc[~pf_ratio_is_na, 'po2'] / df.loc[~pf_ratio_is_na, 'fio2']
df.loc[~pf_ratio_is_na, 'pf_ratio'] = df.loc[~pf_ratio_is_na, 'pf_ratio'].map(lambda x: int(round(x * 100)))

In [ ]:
df['nice_morbid_obesity'] = df['bmi'].map(lambda x: x > 35)
df.nice_morbid_obesity.value_counts()

In [ ]:
from data_warehouse_utils.dataloader import DataLoader

dl = DataLoader()
df_aki = dl.get_patients()
df_aki = df_aki[['hash_patient_id', 'nice_aki']]
df = pd.merge(df, df_aki, how='left', on='hash_patient_id')


In [ ]:
print(len(df.index))
df[['fio2', 'peep', 'po2', 'pf_ratio']].describe()


In [ ]:
from data_warehouse_utils.dataloader import DataLoader

dl = DataLoader()
patients = df['hash_patient_id'].unique().tolist()
df_rotation = dl.get_range_measurements(patients=patients,
                                        parameters=['position'])

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df_rotation.to_csv('rotation_data.csv', index=False)

In [ ]:
df_rotation = df_rotation.loc[(df_rotation.effective_value == '30_degrees') |
                              (df_rotation.effective_value == '45_degrees') |
                              (df_rotation.effective_value == 'bed_chair'),
                              ['start_timestamp', 'hash_patient_id']]

In [ ]:
df_rotation = df_rotation.loc[(df_rotation.effective_value == '30_degrees'), ['start_timestamp', 'hash_patient_id']]

In [ ]:
df_rotation.head()

In [ ]:
df_rotation.info()

In [ ]:
def was_rotated(x, y, z, df):
    mask = (y < df.start_timestamp) & (df.start_timestamp < z) & (df.hash_patient_id == x)
    df = df[mask].sort_values(by=['start_timestamp'])
    # return True if len(df.index) == 0 else False
    # return len(df.index)
    return 0 if len(df.index) == 0 else df.start_timestamp.iloc[0] - y

In [ ]:
rotation_list = [was_rotated(x, y, z, df_rotation) for x, y, z in
            zip(df.loc[:, 'hash_patient_id'],
                df.loc[:, 'start_timestamp'],
                df.loc[:, 'end_timestamp'])]

In [ ]:
error = []

for i in range(len(rotation_list)):
    if rotation_list[i] == 0:
        error.append(0)
    else:
        error.append(int(round(rotation_list[i].total_seconds()/(60*60))))

In [ ]:
df['error'] = error

In [ ]:
df['error'].describe()

In [ ]:
df.info(max_cols=200)

In [ ]:
df.head()

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df.to_csv('data_raw.csv', index=False)

In [ ]:
df.loc[df.treated & (df.error > 0) & (df.error < df.duration_hours), 'error'].describe()

In [ ]:
df.hash_patient_id.nunique()

In [ ]:
COLUMNS_TO_DROP = ['hash_session_id',
                   'hash_patient_id',
                   'start_timestamp',
                   'end_timestamp',
                   'duration_hours',
                   'pacmed_origin_hospital',
                   'fio2_inclusion_8h',
                   'peep_inclusion_8h',
                   'po2_inclusion_8h',
                   'artificial_session',
                   'death_timestamp',
                   'outcome',
                   'mortality',
                   'icu_mortality']
df.drop(columns=COLUMNS_TO_DROP, inplace=True)
df.info(max_cols=200)

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df.to_csv('data_all_variables.csv', index=False)

In [ ]:
VARIABLES_RCT = ['treated',
                 'age',
                 'gender',
                 'bmi',
                 'nice_diabetes',
                 'nice_aki',
                 'nice_cirrhosis',
                 'nice_hem_malign',
                 'nice_copd',
                 'nice_imm_insuf',
                 'nice_cardio_vasc_insuf',
                 'nice_morbid_obesity',
                 'renal_replacement_therapy',
                 'sofa_score',
                 'lactate',
                 'tidal_volume',
                 'tidal_volume_per_kg',
                 'respiratory_rate_measured',
                 'peep',
                 'fio2',
                 'po2',
                 'pco2',
                 'ph',
                 'plateau_pressure',
                 'driving_pressure',
                 'lung_compliance_static',
                 'med_muscle_relaxants',
                 'med_vasopressors',
                 'med_glucocorticoids',
                 'pf_ratio',
                 'pf_ratio_2h_8h_outcome',
                 'pf_ratio_2h_8h_manual_outcome',
                 'pf_ratio_12h_24h_outcome',
                 'pf_ratio_12h_24h_manual_outcome']

df_rct = df[VARIABLES_RCT]
df_rct.info(max_cols=200)

#Should add a late outcome 18h-22h

In [ ]:
df_rct.head()


In [ ]:
df_rct.info()

In [ ]:
df_rct = df_rct.drop(columns=['plateau_pressure'])

In [ ]:
os.chdir('/home/adam/adam/data/19012021/')
df_rct.to_csv('data_guerin_rct.csv', index=False)

In [ ]:
df_rct.describe()

In [ ]:

df_rct.info()